<a href="https://colab.research.google.com/github/prajeeth15/Automated-e-mail-reply-by-chatbot-using-pytorch-Neural-Networks-/blob/main/email_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):
    """
    split sentence into array of words/tokens
    a token can be a word or punctuation character, or number
    """
    return nltk.word_tokenize(sentence)


def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag


In [ ]:
import torch
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [ ]:
import numpy as np
import random
import json
import nltk
nltk.download('punkt')
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader



with open('intents.json', 'r') as f:

    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 20
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
26 patterns
7 tags: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
54 unique stemmed words: ["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']
54 7
Epoch [100/1000], Loss: 0.0565
Epoch [200/1000], Loss: 0.0098
Epoch [300/1000], Loss: 0.0012
Epoch [400/1000], Loss: 0.0033
Epoch [500/1000], Loss: 0.0010
Epoch [600/1000], Loss: 0.0005
Epoch [700/1000], Loss: 0.0011
Epoch [800/1000], Loss: 0.0001
Epoch [900/1000], Loss: 0.0001
Epoch [1000/1000], Loss: 0.0003
final loss: 0.0003
train

# New Section

In [ ]:
#@title Example form fields
#@markdown Forms support many types of fields.

no_type_checking = ''  #@param
string_type = 'example'  #@param {type: "string"}
slider_value = 143  #@param {type: "slider", min: 100, max: 200}
number = 102  #@param {type: "number"}
date = '2010-11-05'  #@param {type: "date"}
pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
#@markdown ---


In [ ]:
import random
import json

import torch




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

regex = '^(\w|\.|\_|\-)+[@]vitstudent.ac.in'

bot_name = "Faculty"
print("Mail your query! (type 'quit' to exit)\n")
while True:
    emailid=input("enter the mail id: ")
    pattern="vitstudent.ac.in"
    if(pattern in emailid):
      username=input("Username: ")
      password=input("Password: ")
      sentence = input("Student: ")
      if sentence == "quit":
         break
    
    
      sentence = tokenize(sentence)
      X = bag_of_words(sentence, all_words)
      X = X.reshape(1, X.shape[0])
      X = torch.from_numpy(X).to(device)

      output = model(X)
      _, predicted = torch.max(output, dim=1)

      tag = tags[predicted.item()]

      probs = torch.softmax(output, dim=1)
      prob = probs[0][predicted.item()]
      if prob.item() > 0.75:
          for intent in intents['intents']:
              if tag == intent["tag"]:
                  print(f"{bot_name}: {random.choice(intent['responses'])}")
      else:
          print(f"{bot_name}: Please contact me after 7:00 pm today")
    else:
       print("Only VIT students allowed to ask the queries!")



Mail your query! (type 'quit' to exit)

enter the mail id: mj@vitstudent.ac.in
Username: ghhj
Password: fghhh
Student: Sir, can I come to your cabin to rectify my doubts regarding the CAT marks sir
Faculty: Please contact me after 7:00 pm today


In [ ]:
{       
     "intents": [   

       {    
         "tag": "lab class redo",   
         "patterns": [  
           "Sir unfortunately i was not able to attend the last week lab class so what should i do for that?",  
           "Sir due to fault in the instrument my readings for the last experiment done in last week lab is not giving any meaningful values so i would like to redo the experiment",   
           "Sir I missed the last lab class so would like to redo it within next week",
"Sir i would like to perform to redo the last week experiment just to get more familiar with instrument"    
         ], 
         "responses": [ 
           "Dear student,Please contact contact the lab in charge check for the free slots for lab and try to complete the experiment within the free slot" 
         ]
},  
       {    
         "tag": "attendance not marked",    
         "patterns": ["Sir even though i attend the last lecture i was marked absent can you please rectify this", "Sir i have been marked absent for the last week classes though i attended all of them so can you change this as this would affect my overall attendance", "Sir as biometric was not working i could not give my attendance for last class please consider my attendance "], 
         "responses": ["Dear student, recently i received the updated attendance list, will mark your attendance accordingly",
            "Dear student please contact PAT office, none of your biometric attendance has been recorded"]  
       },
    
       {    
         "tag": "Attendance less than 75",  
         "patterns": [  
           "Sir I need attendance for one class so that my attendance reaches 75 percentage and make me eligible to write Final examination.",  
           "Sir I require attendance for two classes to increase my attendance to 75 percentage and help me write the final exam",  
        "Sir I am running short on one class for getting 75 percentage attendance and help me attend final examination."    
         ], 
         "responses": [ 
           "Dear student,Please contact the Head of Department and get his approval in the form of an approval letter with his signature and stamp, and I will consider."   
         ]  
    },  
       {    
         "tag": "Lab Class",    
         "patterns": [  
           "Sir I missed the previous lab class and I want to redo the experiment done on that particular class. Kindly do the needful.",   
           "Sir as I missed the lab class last month and you asked me to remind you about redoing that experiment.",    
        "Sir I missed the last weeks lab class and I need to redo the experiment done on that class. When can I do this experiment?"    
         ], 
         "responses": [ 
           "Dear student,Please contact the Lab staff and ask when the lab will be free and do the lab experiment and complete it"  
         ]  
    },  
        
       {    
         "tag": "CAT File not uploaded",    
         "patterns": [  
           "Sir I was not able to upload the answer sheet for the cat exam conducted last week. I have mailed you the answer sheet with this mail. Kindly do the needful.", 
           "Sir as I faced some issue while uploading the answer script So I have sent you the answer script along with this mail as you instructed. Please look into this.",   
        "CAT answer script attached."   
         ], 
         "responses": [ 
           "Dear student, I have received your response. But marks will be deducted if it is submitted after deadline"
        ]   
            
        
    },  
   {

    "tag": "urgent sir",
     "patterns": [
    "Urgent Sir, I missed my quiz today as i overslept , please consider requiz.",
    "sir its an urgent mail, I was present in yesterday's class but i am marked as absent , please do the needfull sir.",
    "Urgent! Sir i couldnt upload the answer script of one question in the cat exam on codetantra due to network issues, can i send you that answer script to your mail sir?"],
"responses":[
"Dear student, its your responsibility to attend the quiz at the right time. This time i will consider but make sure you do not repeat it again."
 "Ok student! I will make the changes.",
 "Please contact the Hod pa, we are not responsibile for this."
]
},

    {

    "tag": "class doubt regards",
    "patterns": [
    "Sir, I have some doubts in the topic that you took in today’s class. When can I contact you to rectify my doubts sir",
"Sir, I have doubt in the topic that has been give in DA. Can u please rectify my doubt in next class sir",
"Sir, I have doubts in the topics that has been included in CAT syllabus. Can you please conduct one doubt session to clear all our doubts sir"],
"responses":[
"Dear student, let us meet in next class and I will clear all your doubts",
"Dear student, meet me in my cabin before lunch to rectify your doubts",
"Dear student, I have uploaded the class materials. Go through that once for clear understanding of the topic"
]
},
    
       {    
         "tag" : "reCAT",   
         "patterns": [  
           "Sir Owing to my bad health for the past month i was missed my classes and CAT exams so i request you to conduct my Cat exam again", 
           "Sir I had Onduty for the past few weeks due to which i missed my CAT exams. Please consider conducting ReCAT exam", 
           "Sir due to bad internet connection i could not attend my CAT exams so i would like you conduct ReCAT exams at the earliest",
           "sir I missed my CAT exams last week as i had a family function to attend please consider reconducting reCAT"    
         ], 
         "responses": [ 
           "Dear student,Please contact the Head of Department and get his approval in the form of an approval letter with his signature and stamp, and I will consider possibilities of reCAT" 
         ]  
       },   
            
       {    
         "tag": "Revaluation FAT",  
         "patterns": [  
           "Sir i failed in my FAT exam by one mark so would you consider evaluating my answer script", 
           "Sir i have applied for revaluation for FAT so will you evaluate my answer script for increasing my marks",  
           "Sir even after revaluation my FAT marks has not been increased will consider you evaluating my answer script"
            
         ], 
         "responses": [ 
           "Dear student,I am ready to check your answer script but i can not change your final marks",
           "Dear student,I am not control of evaluating your answer script please consider contacting Head of Department"   
            
         ]  
       },
       {    
        "tag": "DA question paper uploaded",
   
        "patterns": [   
   "Sir, the last upload for DA is next week and the Digital Assignment question paper has not been uploaded in the VTOP yet. Can u please upload it or extend the date sir?",
"Sir the due date for the Digital assignment is next week  and the question paper has not been uploaded. Sir could you please upload it at the earliest as it will be easier for us to complete it well within the deadline",
"Sir with the deadline for the Digital assignment nearing, I happened to notice that the question paper hasn’t been uploaded yet. Sir with due respect could you please upload it at the earliest or extend the due date to make things easier for us?",    
   "Sir can you please send the DA question paper  by mail as in the Vtop we were not able to download it." 
       
       
       
       
        ],
   
        "responses": [  
          "Dear student, I will try to upload the DA question paper by tonight and don’t worry about the deadline as it won’t take much time for completing it",    
          "Dear student, Can u check VTOP again, I have already uploaded the question paper, if it is still no showing I will mail u the question paper",   
          "Dear student, we will discuss this on our next class and don’t worry about the deadline",    
          "Dear student, I will try to upload it as soon as possible. If you are not able to do it on time, I will accept the DA by mail, so don’t worry"   
        ]
    },
    {
        "tag": "re-quiz",
        "patterns": [
        "Sir I deeply regret to say that i missed the quiz due to some unfortunate reasons. I request you to kindly consider this as an apology and conduct a re-quiz for me on any day that suits you",
        "Sir, due to some network connectivity issues in my area I was not able to submit my quiz. Can u please conduct re-quiz for me sir",
        "Sir as I had OD(on duty) on that day I was not able to attend the quiz. Can u please consider me for re-quiz sir"],
        "responses":[
        "Dear student, I will discuss this in the class as many had this issue and we will plan a re-quiz accordingly",
        "Dear student, if your reason is legitimate I will consider you for re-quiz. Lets discuss about this in class"
        ]
        },

        {
            "tag": "quiz marks regards",
            "patterns":[
            "Sir, I have some doubts regarding the quiz marks that has been uploaded in VTOP. Can I please see my paper once sir",
            "Sir, quiz marks has not been updated in VTOP for me, can you please check that once sir",
            "Sir, can I come to your cabin to rectify my doubts regarding the quiz questions and marks sir",
            "Sir, can you please consider my late submission and evaluate my quiz paper sir."],
            "responses": [
            "Dear student, there won’t  be any change in the quiz marks that has been uploaded in VTOP. Check the answer key that has been uploaded in VTOP once",
            "Dear student, I will distribute the quiz papers in next class and I will discuss the answer key with you",
            "Dear student, quiz answer key is uploaded in VTOP. Check that once and contact me if your answer is not marked according to key"]
            },
            
            {
                "tag": "cat marks regards",
                "patterns":[
                "Sir, I have some doubts regarding the CAT marks that has been uploaded in VTOP. Can I please see my paper once sir",
                "Sir, CAT marks has not been updated in VTOP for me, can you please check that once sir",
                "Sir, can I come to your cabin to rectify my doubts regarding the CAT marks sir",
                "Sir, can u please reconsider my CAT paper once sir"],
                "responses": [
                "Dear student, the marks that have been uploaded are final and I won’t make any changes. If you still have doubts, meet me after lunch at my cabin",
                "Dear student, I will distribute the CAT papers in next class and I will discuss the answer key with you",
                "Dear student, CAT answer key is uploaded in VTOP. Check that once and contact me if your answer is not marked according to key"]
                }
                
]
}
